In [20]:
%pip install pyarrow dask[complete] dask-ml

Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [1]:
links = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet", 
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"]

In [4]:
df_list = [
    pd.read_parquet(i) for i in links
]

In [23]:
df = pd.concat(df_list, axis=0).reset_index(drop=True)

## Question 1

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [24]:
df_list[0].columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [25]:
len(df_list[0].columns)

19

In [26]:
len(df_list[1].columns)

19

## Question 2

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [35]:
sec_ride = (df_list[0]["tpep_dropoff_datetime"] - df_list[0]["tpep_pickup_datetime"]).dt.total_seconds()

In [37]:
sec_ride.std()/60

42.59435124195673

## Question 3
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [39]:
sec_ride_all = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds()

In [49]:
in_lier_df = df_list[0][(sec_ride >= 60) & (sec_ride <= 3600)]

In [50]:
len(in_lier_df)/len(df_list[0])

0.9812202822125979

## Question 4

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [51]:
df_list[0]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [5]:
train_df = df_list[0][['PULocationID', 'DOLocationID']]

In [32]:
len(pd.concat([train_df['PULocationID'], train_df['DOLocationID']], axis=0).unique())

262

In [35]:
%pip install scikit-learn==1.4.2

Note: you may need to restart the kernel to use updated packages.


In [7]:
from sklearn.feature_extraction import DictVectorizer

train_df['PULocationID'] = train_df['PULocationID'].astype(str)
train_df['DOLocationID'] = train_df['DOLocationID'].astype(str)

data_dicts = train_df.to_dict(orient='records')

vectorizer = DictVectorizer(sparse=True)

feature_matrix = vectorizer.fit_transform(data_dicts)

dimensionality = feature_matrix.shape[1]

print(dimensionality)

/tmp/ipykernel_18153/782199142.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['PULocationID'] = train_df['PULocationID'].astype(str)
/tmp/ipykernel_18153/782199142.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['DOLocationID'] = train_df['DOLocationID'].astype(str)


518


## Question 5
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [10]:
target = sec_ride = (df_list[0]["tpep_dropoff_datetime"] - df_list[0]["tpep_pickup_datetime"]).dt.total_seconds()/60

In [12]:
target.values

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
model = LinearRegression()
model.fit(feature_matrix, target.values)
predictions = model.predict(feature_matrix)
rmse = np.sqrt(mean_squared_error(target, predictions))
print(f'The RMSE on the training data is: {rmse:.2f}')

The RMSE on the training data is: 42.00


## Question 6
Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [16]:
test_df = df_list[1][['PULocationID', 'DOLocationID']]
test_df['PULocationID'] = test_df['PULocationID'].astype(str)
test_df['DOLocationID'] = test_df['DOLocationID'].astype(str)

data_dicts = test_df.to_dict(orient='records')

test_feature_matrix = vectorizer.transform(data_dicts)

/tmp/ipykernel_18153/2797695480.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['PULocationID'] = test_df['PULocationID'].astype(str)
/tmp/ipykernel_18153/2797695480.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['DOLocationID'] = test_df['DOLocationID'].astype(str)


In [17]:
test_target = (df_list[1]["tpep_dropoff_datetime"] - df_list[1]["tpep_pickup_datetime"]).dt.total_seconds()/60

In [18]:
predictions = model.predict(test_feature_matrix)
rmse = np.sqrt(mean_squared_error(test_target, predictions))
print(f'The RMSE on the test data is: {rmse:.2f}')

The RMSE on the test data is: 42.25
